In [34]:
# source /cvmfs/sft.cern.ch/lcg/views/LCG_104/x86_64-centos7-gcc11-opt/setup.sh
import numpy as np
import ROOT
import math
import h5py
import xml.etree.ElementTree as ET

In [35]:
# Read the file and make the tree
f = ROOT.TFile.Open("../temp/xaodanahelpersminimalexample/data/trees/user.mmcohen.data22_13p6TeV_AOD_EB_1-24-2024_tree.root/user.mmcohen.00440499.37026504._000001.tree.root")
#f = ROOT.TFile.Open("../temp/xaodanahelpersminimalexample/data/trees/EB_v1_1-20-2024_-1/data-tree/data22_13p6TeV.00440499.physics_EnhancedBias.merge.AOD.r15242_r15243_p6010_tid36850548_00.root")

td = f.Get("EB_Tree")
tree = td.Get("nominal")
nevents = tree.GetEntries() # number of events
print(nevents)

1019220


In [36]:
# Maximum numbers of each object
NUM_JETS = 10
NUM_ELECTRONS = 3
NUM_MUONS = 3
NUM_PHOTONS = 3

In [37]:
# Min/max pt and eta values
MIN_JET_PT = 0
MIN_PHELMU_PT = 0
MAX_ETA = 1000

In [38]:
# List of triggers that we want to keep L1 events from for HLT training
L1_save_trigs = [
    'L1_3J35p0ETA23',
    'L1_3J50',
    'L1_4J15',
    'L1_4J15p0ETA25',
    'L1_4J20',
    'L1_5J15p0ETA25',
    'L1_J85_3J30',
    'L1_HT150-J20s5pETA31_MJJ-400-CF',
    'L1_HT190-J15s5pETA21',
    'L1_SC111-CJ15',
    'L1_DR-TAU20ITAU12I-J25',
    'L1_TAU100',
    'L1_TAU20IM_2TAU12IM_4J12p0ETA25',
    'L1_TAU25IM_2TAU20IM_2J25_3J20',
    'L1_TAU60_2TAU40',
    'L1_TAU60_DR-TAU20ITAU12I',
    'L1_2MU8F',
    'L1_3MU3VF',
    'L1_3MU5VF',
    'L1_4MU3V',
    'L1_MU10BOM',
    'L1_MU12BOM',
    'L1_MU14FCH',
    'L1_MU5VF_3MU3VF',
    'L1_MU8VF_2MU5VF',
    'L1_BPH-0M9-EM7-EM5_2MU3V',
    'L1_2EM8VH_MU8F',
    'L1_2J15_XE55',
    'L1_3J15p0ETA25_XE40',
    'L1_EM15VHI_2TAU12IM_4J12',
    'L1_EM15VHI_2TAU12IM_XE35',
    'L1_EM15VH_MU8F',
    'L1_MU8F_TAU12IM_3J12',
    'L1_MU8F_TAU12IM_XE35',
    'L1_MU8F_TAU20IM',
    'L1_TAU40_2TAU12IM_XE40',
    'L1_MJJ-500-NFF',
    'L1_MU18VFCH',
    'L1_J45p0ETA21_3J15p0ETA25',
    'L1_MU8F_2J15_J20',
    'L1_2EM20VH',
    'L1_XE55',
    'L1_EM18VHI_MJJ-300',
    'L1_2EM15VHI',
    'L1_J120',
    'L1_EM20VH_3EM10VH',
    'L1_MU5VF_EMPTY',
    'L1_J40p0ETA25_2J25_J20p31ETA49',
    'L1_2J50_XE40',
    'L1_J25p0ETA23_2J15p31ETA49',
    'L1_2MU5VF_3MU3V'
]

In [39]:
def sort_and_pad(in_data, max_objects):
    """input an object of size (4, num_objects), as well as the maximum number of objects wanted.
    in_data[0] corresponds to the pt array, and the other three are E, eta, phi."""
    
    indices = np.argsort(in_data[0])[-max_objects:][::-1] # indices that sort the in_data by pt
    arrs = []
    for array in in_data:

        # Sorting
        sorted_arr = array[indices]

        # Padding
        if len(array) < max_objects:
            sorted_arr = np.concatenate((sorted_arr, np.zeros(max_objects - len(array))))

        arrs.append(sorted_arr)
        
    
    arrs = np.array(arrs).T
    return np.array(arrs)

In [40]:
def make_mask(min_pt, max_eta, pt, eta):
    """Make a mask on pt and eta of event, in case I want to make any data cuts."""

    return [(pt > MIN_JET_PT) and (abs(eta) < MAX_ETA) for pt, eta in zip(pt, eta)]

In [41]:
def get_weights(events_lumiblocks, weights_file, run_file):
    """Collect the Enhanced Bias weights. Done by 
    reading the weights_file xml file which contains the weights for each event,
    and the run_file xml file, which contains information regarding which
    lumiblocks are bad.
    
    events_lumiblocks: list of tuples [('event_number', 'lumiblock_number'), (), ...], as strings."""

    # Parse the XML files
    weights_tree = ET.parse(weights_file)
    lumi_tree = ET.parse(run_file)

    # Build the weights dictionary
    weights_dict = {weight.get('id'): weight.get('value') for weight in weights_tree.findall('./weights/weight')}

    # Build a dictionary for events to find their weights
    event_weights = {event.get('n'): weights_dict.get(event.get('w')) for event in weights_tree.findall('./events/e')}

    # Build a set of bad lumiblocks
    bad_lumiblocks = {lb.get('id') for lb in lumi_tree.findall('./lb_list/lb[@flag="bad"]')}

    # Process each event-lumiblock pair
    results = []
    for event_number, lumiblock in events_lumiblocks:
        event_weight = event_weights.get(event_number)
        is_lumiblock_bad = lumiblock in bad_lumiblocks
        results.append({
            "event_number": event_number,
            "lumiblock": lumiblock,
            "weight": event_weight,
            "is_lumiblock_bad": is_lumiblock_bad
        })

    return results

In [42]:
def get_data(event, c_name, vars=['pt', 'm', 'eta', 'phi'], mask=None):
    """
    Extracts and processes data from a specified container within an event.

    event: The event object containing the data.
    c_name: Name of the container to get the data from.
    vars: List of the variables to pull from the container.
    mask: The mask to apply to the data, if applicable.
    max_objects: Maximum number of objects to retain after sorting and padding.
    return: Processed data from the specified container, after being sorted and padded.
    """
    data = []
    for var in vars:
        full_var_name = f"{c_name}_{var}"
        var_data = getattr(event, full_var_name)

        if mask is not None:
            masked_data = np.asarray([x for x, m in zip(var_data, mask) if m])
        else:
            masked_data = np.asarray([x for x in var_data]) # for some reason this syntax runs way faster!?

        data.append(masked_data)
    return data
    #return sort_and_pad(data, max_objects=max_objects)

# Usage example:
# jet_data = get_data(event, 'HLT_jet', vars=['pt', 'E', 'eta', 'phi'], mask=jet_mask, max_objects=NUM_JETS)

In [94]:
HLT_jet_list = []
el_list = []
LRT_el_list = []
muon_list = []
LRT_muon_list = []
ph_list = []
MET_list = []
trig_list = []
ev_lb_list = []
pass_L1_unprescaled = []

for i, event in enumerate(tree):
    
    # Collect event number and lumiblock for future weight calculation
    ev_num = str(event.eventNumber)
    lb_num = str(event.lumiBlock)
    ev_lb_list.append((ev_num, lb_num))

    # Masks
    # HLT_jet_mask = make_mask(min_pt=MIN_JET_PT, max_eta=MAX_ETA, pt=event.HLT_jet_pt, eta=event.HLT_jet_eta)
    # el_mask = make_mask(min_pt=MIN_PHELMU_PT, max_eta=MAX_ETA, pt=event.HLT_el_pt, eta=event.HLT_el_eta)
    # el_LRT_mask = make_mask(min_pt=MIN_PHELMU_PT, max_eta=MAX_ETA, pt=event.HLT_el_LRT_pt, eta=event.HLT_el_LRT_eta)
    # muon_mask = make_mask(min_pt=MIN_PHELMU_PT, max_eta=MAX_ETA, pt=event.HLT_muon_pt, eta=event.HLT_muon_eta)
    # muon_LRT_mask = make_mask(min_pt=MIN_PHELMU_PT, max_eta=MAX_ETA, pt=event.HLT_muon_LRT_pt, eta=event.HLT_muon_LRT_eta)
    # ph_mask = make_mask(min_pt=MIN_PHELMU_PT, max_eta=MAX_ETA, pt=event.ph_pt, eta=event.ph_eta)

    # HLT Jets
    HLT_jet_data = get_data(event, 'HLT_jet', vars=['pt', 'E', 'eta', 'phi'])
    # HLT_jet_data = get_data(event, 'HLT_jet', vars=['pt', 'E', 'eta', 'phi'], mask=jet_mask)
    HLT_jet_list.append(sort_and_pad(HLT_jet_data, max_objects=NUM_JETS))

    # Electrons
    el_data = get_data(event, 'HLT_el', vars=['pt', 'm', 'eta', 'phi'])
    el_list.append(sort_and_pad(el_data, max_objects=NUM_ELECTRONS))

    # LRT Electrons
    LRT_el_data = get_data(event, 'HLT_el_LRT', vars=['pt', 'm', 'eta', 'phi'])
    LRT_el_list.append(sort_and_pad(LRT_el_data, max_objects=NUM_ELECTRONS))

    # Muons
    muon_data = get_data(event, 'HLT_muon', vars=['pt', 'm', 'eta', 'phi'])
    muon_list.append(sort_and_pad(muon_data, max_objects=NUM_MUONS))

    # LRT Muons
    LRT_muon_data = get_data(event, 'HLT_muon_LRT', vars=['pt', 'm', 'eta', 'phi'])
    LRT_muon_list.append(sort_and_pad(LRT_muon_data, max_objects=NUM_MUONS))
    
    # Photons
    ph_data = get_data(event, 'ph', vars=['pt', 'm', 'eta', 'phi'])
    ph_list.append(sort_and_pad(ph_data, max_objects=NUM_PHOTONS))

    # MET
    MET_data = [np.float32(event.trigmetMet), 0, 0, np.float32(event.trigmetMetPhi)]
    MET_list.append(MET_data)

    # Trigger Decisions
    trig_data = [str(trigger) for trigger in event.passedTriggers]
    trig_list.append(trig_data)

    # Check if the event passed any of the unprescaled physics triggers from the list:
    pass_L1_unprescaled.append(1*any(trigger in L1_save_trigs for trigger in trig_data)) # 0=no 1=yes
    

    # Print progress
    if (i % 10000) == 1:
        print(f'Progress: {i} / {nevents}')

Progress: 1 / 1019220
Progress: 10001 / 1019220
Progress: 20001 / 1019220
Progress: 30001 / 1019220
Progress: 40001 / 1019220
Progress: 50001 / 1019220
Progress: 60001 / 1019220
Progress: 70001 / 1019220
Progress: 80001 / 1019220
Progress: 90001 / 1019220
Progress: 100001 / 1019220
Progress: 110001 / 1019220
Progress: 120001 / 1019220
Progress: 130001 / 1019220
Progress: 140001 / 1019220
Progress: 150001 / 1019220
Progress: 160001 / 1019220
Progress: 170001 / 1019220
Progress: 180001 / 1019220
Progress: 190001 / 1019220
Progress: 200001 / 1019220
Progress: 210001 / 1019220
Progress: 220001 / 1019220
Progress: 230001 / 1019220
Progress: 240001 / 1019220
Progress: 250001 / 1019220
Progress: 260001 / 1019220
Progress: 270001 / 1019220
Progress: 280001 / 1019220
Progress: 290001 / 1019220
Progress: 300001 / 1019220
Progress: 310001 / 1019220
Progress: 320001 / 1019220
Progress: 330001 / 1019220
Progress: 340001 / 1019220
Progress: 350001 / 1019220
Progress: 360001 / 1019220
Progress: 37000

In [95]:
weights_file = '/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/TrigCostRootAnalysis/EnhancedBiasWeights_440499.xml'
weights_run_file = '/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/TrigCostRootAnalysis/enhanced_bias_run_440499.xml'

In [96]:
# Collect weights and lumiBlock information
results = get_weights(events_lumiblocks=ev_lb_list, weights_file=weights_file, run_file=weights_run_file)
weights_list = [np.float64(entry['weight']) for entry in results]
lumiblock_bads_list = [entry['is_lumiblock_bad'] for entry in results]

In [97]:
print(ev_lb_list[0:5])
print(weights_list[0:10])
print(lumiblock_bads_list[0:5])

[('1309931188', '511'), ('1309979677', '511'), ('1309927610', '511'), ('1309926471', '511'), ('1309928291', '511')]
[10.0, 479349.0, 12.0, 629.758, 453438.0, 105517.0, 137518.0, 39291.1, 453438.0, 137518.0]
[False, False, False, False, False]


In [98]:
print(len(weights_list))
print(weights_list[0:5])

1019220
[10.0, 479349.0, 12.0, 629.758, 453438.0]


# ___________________

In [99]:
with h5py.File('../ntuples/AOD_EB_ntuples_1-26-2024.h5', 'w') as hf:
    hf.create_dataset("HLT_jets", data=np.array(HLT_jet_list))
    hf.create_dataset("electrons", data=np.array(el_list))
    hf.create_dataset("LRT_electrons", data=np.array(LRT_el_list))
    hf.create_dataset("muons", data=np.array(muon_list))
    hf.create_dataset("LRT_muons", data=np.array(LRT_muon_list))
    hf.create_dataset("photons", data=np.array(ph_list))
    hf.create_dataset("MET", data=np.array(MET_list))
    #hf.create_dataset("trig", data=np.array(trig_list))
    #hf.create_dataset("trigger_names", data=trigger_names)
    hf.create_dataset("pass_L1_unprescaled", data=np.array(pass_L1_unprescaled))
    hf.create_dataset("EB_weights", data=weights_list)
    hf.create_dataset("lumiblock_bads", data=lumiblock_bads_list)

In [100]:
# Read file
with h5py.File('../ntuples/AOD_EB_ntuples_1-26-2024.h5', 'r') as hf:
    jets = hf['HLT_jets'][:]
    electrons = hf['electrons'][:]
    LRT_electrons = hf['LRT_electrons'][:]
    muons = hf['muons'][:]
    LRT_muons = hf['LRT_muons'][:]
    photons = hf['photons'][:]
    MET = hf['MET'][:].reshape(-1, 1, 4)  # Broadcasting MET
    pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
    EB_weights = hf["EB_weights"][:]

In [101]:
print(f'jets: {jets.shape}')
print(f'electrons: {electrons.shape}')
print(f'LRT_electrons: {LRT_electrons.shape}')
print(f'muons: {muons.shape}')
print(f'LRT_muons: {LRT_muons.shape}')
print(f'photons: {photons.shape}')
print(f'MET: {MET.shape}')
print(f'pass_L1_unprescaled: {pass_L1_unprescaled.shape}')
print(f'EB_weights: {EB_weights.shape}')

jets: (1019220, 10, 4)
electrons: (1019220, 3, 4)
LRT_electrons: (1019220, 3, 4)
muons: (1019220, 3, 4)
LRT_muons: (1019220, 3, 4)
photons: (1019220, 3, 4)
MET: (1019220, 1, 4)
pass_L1_unprescaled: (1019220,)
EB_weights: (1019220,)


In [112]:
print(f'rejection: {1 / (np.sum(EB_weights * pass_L1_unprescaled) / np.sum(EB_weights))}')

rejection: 609.5950386851063


## ____________________

In [34]:
# MET bias

# MET: 0 --> 0.001 and -999 --> 0
MET_zeros = data[:, 19, 0] == 0
MET_999 = data[:, 19, 0] == -999
data[MET_zeros, 19, 0] = 0.001
data[MET_999, 19, :] = 0

In [35]:
non_zero_rows = np.any(data != 0, axis=(1, 2))
data = data[non_zero_rows, :, :]
trig = trig[non_zero_rows]
print(f'data: {data.shape}')
print(f'trigger decisions: {trig.shape}')

data: (1864687, 26, 4)
trigger decisions: (1864687, 7)
